In [119]:
# import the required libraries
import pandas as pd
from fuzzywuzzy import fuzz


In [120]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_pred = folderlink+input_folder+"new_predikanten.csv"

In [121]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [122]:
jaar_vertrek_type = {'jaar vertrek': pd.Int64Dtype(), 'ind_id': pd.Int64Dtype(),'dag intrede': pd.Int64Dtype(), 'dag vertrek': pd.Int64Dtype() }
df = pd.read_csv(input_pred, sep=',', dtype= jaar_vertrek_type, encoding='utf-8')

In [123]:
df.head()

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN


In [124]:
sim_df = df.head(50)
sim_field = 'predikant'

In [125]:
# Function to calculate string similarity
def calculate_similarity(name1, name2):
    return fuzz.token_set_ratio(name1, name2)

# Unique names for the matrix
unique_names = sim_df[sim_field].drop_duplicates()

# Calculate similarity matrix only for unique names
similarity_matrix = pd.DataFrame(index=unique_names, columns=unique_names)
for name1 in unique_names:
    for name2 in unique_names:
        similarity_matrix.loc[name1, name2] = calculate_similarity(name1, name2)

# Set a similarity threshold to identify possibly matching individuals
similarity_threshold = 50

# Create a new column for matching IDs
sim_df['Matching_ID'] = ''

# Assign matching IDs
current_id = 1
for name in unique_names:
    # If this unique name has already been assigned an ID, skip it
    if sim_df[sim_df[sim_field] == name]['Matching_ID'].iloc[0] != '':
        continue

    matching_ids = [name]
    for potential_match in similarity_matrix.columns:
        if similarity_matrix.loc[name, potential_match] >= similarity_threshold:
            matching_ids.append(potential_match)

    # Assign the current ID to all matching (and duplicate) names
    for match in matching_ids:
        sim_df.loc[sim_df[sim_field] == match, 'Matching_ID'] = current_id
        
    current_id += 1

C:\Windows\Temp\ipykernel_14096\2848079839.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sim_df['Matching_ID'] = ''


In [126]:
sim_df.head(100)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,Matching_ID
0,35651,1,ZH,NaN,Oudewater,NaN,(Hekendorp) Brunner; L.,hulppredikant Oirschot,<NA>,NaN,1867,Lobith,<NA>,NaN,1868,NaN,1
1,45546,2,GE,NaN,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,NaN,2
2,7179,3,NB,NaN,Breda,NaN,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN,3
3,21505,3,GR,NaN,Hornhuizen,NaN,Aa; dr. P.J.B.K. Simon van der,kandidaat,<NA>,NaN,1862,Goutum,<NA>,NaN,1866,NaN,3
4,36887,3,NH,NaN,Purmerend,NaN,Aa; dr. P.J.B.K. Simon van der,Goutum,<NA>,NaN,1886,Winterswijk,<NA>,NaN,1892,NaN,3
5,50146,3,GE,NaN,Winterswijk,NaN,Aa; dr. P.J.B.K. Simon van der,Purmerend,<NA>,NaN,1892,Breda,<NA>,NaN,1893,NaN,3
6,15179,3,FR,NaN,Goutum,NaN,Aa; dr. P.J.B.K. van der,Kloosterburen,<NA>,NaN,1866,Purmerend,<NA>,NaN,1886,NaN,3
7,7204,3,NB,NaN,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,NaN,3
8,19779,4,GE,NaN,Herwijnen,NaN,Aa; W.P. van der,kandidaat,24,december,1989,NaN,<NA>,NaN,<NA>,NaN,5
9,35941,5,OV,NaN,Overdinkel,NaN,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,NaN,4
